## Import libraries

In [689]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import os
from scipy.sparse import coo_matrix

## Read Dataset

In [692]:
#read dataset
df = pd.read_csv('amazon.csv')
df.columns

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')

In [693]:
df.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [696]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

In [698]:
#display the shape of the data
df.shape

(1465, 16)

In [700]:
#describe the data
df.describe().T

,count,unique,top,freq
product_id,1465,1351,B07JW9H4J1,3
product_name,1465,1337,"Fire-Boltt Ninja Call Pro Plus 1.83"" Smart Wat...",5
category,1465,211,Computers&Accessories|Accessories&Peripherals|...,233
discounted_price,1465,550,₹199,53
actual_price,1465,449,₹999,120
discount_percentage,1465,92,50%,56
rating,1465,28,4.1,244
rating_count,1463,1143,"9,378",9
about_product,1465,1293,[CHARGE & SYNC FUNCTION]- This cable comes wit...,6
user_id,1465,1194,"AHIKJUDTVJ4T6DV6IUGFYZ5LXMPA,AE55KTFVNXYFD5FPY...",10


# Data Cleaning

## Handling Missing values

In [704]:
#check for any null values
df.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [706]:
# Fill missing values in 'Product Rating' with a default value (e.g., 0)
df['rating_count'] = df['rating_count'].fillna(0)

In [708]:
df.isnull().sum()

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [710]:
#drop unnecessary columns
train_data = df.drop(['user_name', 'review_id', 'review_title', 'review_content', 'product_link'],axis=1)
train_data

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,img_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...",https://m.media-amazon.com/images/W/WEBP_40237...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...",https://m.media-amazon.com/images/W/WEBP_40237...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...",https://m.media-amazon.com/images/W/WEBP_40237...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...",https://m.media-amazon.com/images/I/41V5FtEWPk...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...",https://m.media-amazon.com/images/W/WEBP_40237...
...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,₹379,₹919,59%,4,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...",https://m.media-amazon.com/images/I/41fDdRtjfx...
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,"₹2,280","₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...",https://m.media-amazon.com/images/I/41gzDxk4+k...
1462,B009P2LIL4,Bajaj Majesty RX10 2000 Watts Heat Convector R...,"Home&Kitchen|Heating,Cooling&AirQuality|RoomHe...","₹2,219","₹3,080",28%,3.6,468,International design and styling|Two heat sett...,"AGVPWCMAHYQWJOQKMUJN4DW3KM5Q,AF4Q3E66MY4SR7YQZ...",https://m.media-amazon.com/images/W/WEBP_40237...
1463,B00J5DYCCA,Havells Ventil Air DSP 230mm Exhaust Fan (Pist...,"Home&Kitchen|Heating,Cooling&AirQuality|Fans|E...","₹1,399","₹1,890",26%,4,"8,031",Fan sweep area: 230 MM ; Noise level: (40 - 45...,"AF2JQCLSCY3QJATWUNNHUSVUPNQQ,AFDMLUXC5LS5RXDJS...",https://m.media-amazon.com/images/W/WEBP_40237...


## Handling duplicate Values

In [713]:
#check if duplicates have
train_data.duplicated().sum()

19

## Data Transformation

### Encode Categorical Variables

In [717]:
categorical = ['product_id', 'product_name', 'category', 'about_product', 'user_id']

#display categorical column
le = LabelEncoder()
for col in categorical:
    train_data[col] = le.fit_transform(train_data[col])

train_data[categorical].describe()

,product_id,product_name,category,about_product,user_id
count,1465.000000,1465.000000,1465.000000,1465.000000,1465.000000
mean,678.739932,671.842321,95.152218,647.044369,598.288055
std,386.735334,389.327207,60.595638,376.530507,346.559746
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,346.000000,336.000000,31.000000,322.000000,298.000000
50%,681.000000,672.000000,97.000000,645.000000,605.000000
75%,1011.000000,1010.000000,141.000000,976.000000,900.000000
max,1350.000000,1336.000000,210.000000,1292.000000,1193.000000


In [719]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   int32 
 1   product_name         1465 non-null   int32 
 2   category             1465 non-null   int32 
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1465 non-null   object
 8   about_product        1465 non-null   int32 
 9   user_id              1465 non-null   int32 
 10  img_link             1465 non-null   object
dtypes: int32(5), object(6)
memory usage: 97.4+ KB


## Data Formatting

### Convert Numerical Data columns to float

In [723]:
#Product price should be float.
train_data['actual_price'] = train_data['actual_price'].replace({'₹': '', ',': ''}, regex=True).astype(float)

train_data['discounted_price'] = train_data['discounted_price'].replace({'₹': '', ',': ''}, regex=True).astype(float)

In [725]:
#check all the unique values in the rating column before converting it to a float
sorted(df['rating'].unique(), reverse=False)

['2',
 '2.3',
 '2.6',
 '2.8',
 '2.9',
 '3',
 '3.0',
 '3.1',
 '3.2',
 '3.3',
 '3.4',
 '3.5',
 '3.6',
 '3.7',
 '3.8',
 '3.9',
 '4',
 '4.0',
 '4.1',
 '4.2',
 '4.3',
 '4.4',
 '4.5',
 '4.6',
 '4.7',
 '4.8',
 '5.0',
 '|']

In [727]:
#rating column has special character '|'  which needs to be removed or replaced

#check which row has special character
special_character = train_data[train_data['rating'] == '|']

# replace the special character with NaN
train_data['rating'] = train_data['rating'].replace('|', None)

# Convert the 'rating' column to numeric (this will convert invalid entries to NaN)
train_data['rating'] = pd.to_numeric(train_data['rating'], errors='coerce')

#Fill missing values (NaN) with the average rating
avg_rating = train_data['rating'].mean()
train_data['rating'] = train_data['rating'].fillna(avg_rating)

In [729]:
#convert rating_count colmn to float
train_data['rating_count'] = pd.to_numeric(train_data['rating_count'], errors='coerce')

In [731]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           1465 non-null   int32  
 1   product_name         1465 non-null   int32  
 2   category             1465 non-null   int32  
 3   discounted_price     1465 non-null   float64
 4   actual_price         1465 non-null   float64
 5   discount_percentage  1465 non-null   object 
 6   rating               1465 non-null   float64
 7   rating_count         328 non-null    float64
 8   about_product        1465 non-null   int32  
 9   user_id              1465 non-null   int32  
 10  img_link             1465 non-null   object 
dtypes: float64(4), int32(5), object(2)
memory usage: 97.4+ KB


### Summarization of Categorical Data

In [734]:
# if there are any duplicates in product_id
train_data['product_id'].duplicated().any()

True

In [736]:
# "True" means some rows share same product ids
duplicate_rows = train_data[train_data['product_id'].duplicated(keep=False)]
print(duplicate_rows['product_id'] )

0        346
1        848
2        819
3        643
4        588
        ... 
1007    1262
1010     319
1017    1261
1018    1329
1019     133
Name: product_id, Length: 206, dtype: int32


In [738]:
# Combine duplicates
train_data = train_data.groupby('product_id').agg({
    'product_name': 'first',  # use the first product name
    'category': 'first',
    'discounted_price': 'first',
    'actual_price': 'first',
    'discount_percentage': 'first',
    'rating': 'mean',  # take the average rating
    'rating_count': 'sum',  # sum up the rating counts for duplicates
    'about_product': lambda x: ','.join(x.astype(str)),  # convert to str before joining
    'user_id': lambda x: ','.join(x.astype(str)),  # convert to str before joining
}).reset_index()

# Display the updated DataFrame
print(train_data)

      product_id  product_name  category  discounted_price  actual_price  \
0              0           299        49             507.0        1208.0   
1              1          1073        49             749.0        1339.0   
2              2           319        72             399.0         499.0   
3              3           624        18             279.0         375.0   
4              4           633        18             699.0         995.0   
...          ...           ...       ...               ...           ...   
1346        1346          1149       159             259.0         999.0   
1347        1347           547       136            1299.0        2495.0   
1348        1348           746       159             279.0         499.0   
1349        1349           875        10             249.0         999.0   
1350        1350          1048       140            1439.0        1999.0   

     discount_percentage  rating  rating_count about_product  user_id  
0              

In [740]:
# create new columns: Level 1 to Level 3 Categories
train_data['first_category'] = train_data['category'].astype(str).str.split('|').str[0]
train_data['second_category'] = train_data['category'].astype(str).str.split('|').str[-1]
train_data['third_category'] = train_data['category'].astype(str).str.split('|').str[-2]

In [742]:
# Count the number of products in each category
first_category_counts = train_data['first_category'].value_counts()

# Sort the categories in descending order
first_category_counts = first_category_counts.sort_values(ascending=False)

# Distribution of First categories
pd.DataFrame({'First Category': first_category_counts.index, 'Number of Products': first_category_counts.values})

,First Category,Number of Products
0,10,161
1,117,68
2,119,62
3,97,60
4,76,51
...,...,...
206,111,1
207,152,1
208,150,1
209,0,1


In [744]:
# Indicates the exact number of values
# this code is very useful because it has multiple categorical columns
for col in categorical:
    unique_count = train_data[col].nunique()
    print(f'Number of unique items in {col} column: {unique_count}')

Number of unique items in product_id column: 1351
Number of unique items in product_name column: 1337
Number of unique items in category column: 211
Number of unique items in about_product column: 1297
Number of unique items in user_id column: 1224


# EDA

In [757]:
df['category'].unique()

array(['Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables',
       'Computers&Accessories|NetworkingDevices|NetworkAdapters|WirelessUSBAdapters',
       'Electronics|HomeTheater,TV&Video|Accessories|Cables|HDMICables',
       'Electronics|HomeTheater,TV&Video|Televisions|SmartTelevisions',
       'Electronics|HomeTheater,TV&Video|Accessories|RemoteControls',
       'Electronics|HomeTheater,TV&Video|Televisions|StandardTelevisions',
       'Electronics|HomeTheater,TV&Video|Accessories|TVMounts,Stands&Turntables|TVWall&CeilingMounts',
       'Electronics|HomeTheater,TV&Video|Accessories|Cables|RCACables',
       'Electronics|HomeAudio|Accessories|SpeakerAccessories|Mounts',
       'Electronics|HomeTheater,TV&Video|Accessories|Cables|OpticalCables',
       'Electronics|HomeTheater,TV&Video|Projectors',
       'Electronics|HomeAudio|Accessories|Adapters',
       'Electronics|HomeTheater,TV&Video|SatelliteEquipment|SatelliteReceivers',
       'Computers&Acces